In [1]:
# ! ln 10_prelim_web_app* autoparis_prototype/scripts/
# ! cd autoparis_prototype/ && git add * */* && git commit -a -m "AP update" && git push

In [24]:
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd
from scipy.special import softmax
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table, os, time, glob, numpy as np
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import dash_daq as daq
import json

# sauth --dir web_app_test/ user pass localhost 5555
# //<script src="/openseadragon/openseadragon.min.js"></script>
#     //<script src="/openseadragon/openseadragon-svg-overlay.js"></script>
#     //<script src="/openseadragon/openseadragon-svg-overlay.js"></script>
#     //<script src="/openseadragon/openseadragon-svg-overlay.js"></script>
#     //<script src="/openseadragon/openseadragon-all.js"></script>
#     //<script src="openseadragon-svg-overlay.js"></script> <link rel="stylesheet" href="style.css">
# <script src="/openseadragon/openseadragon.js"></script>
# <script src="/openseadragon/openseadragon-svg-overlay.js"></script>
# https://github.com/openseadragon/svg-overlay/issues/20
# https://stackoverflow.com/questions/14221359/toggle-button-for-d3-js
# points.forEach(function (d) {
#                         var result = viewer1.viewport.imageToViewerElementCoordinates(OpenSeadragon.Point(d[1], d[0]));
#                         d.computedX = result.x;
#                         d.computedY = result.y;
#                     });
# almost there, debug loading of the svg overlay

def replace_all(text, dic):
#     print(dic)
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

return_wsi_file=lambda wsi_name,include_points,json_file,radius,width,height: replace_all("""<html>
<head>
    <title id="title">WSI</title>
    <meta http-equiv="Cache-control" content="no-cache, no-store, must-revalidate">
    <meta http-equiv="Pragma" content="no-cache">
</head>
<body>
    <script src="//code.jquery.com/jquery-1.11.2.min.js"></script>
    <script src="//d3js.org/d3.v5.min.js"></script>
    <script src="/openseadragon/openseadragon.min.js"></script>
    <script src="/openseadragon/openseadragon-svg-overlay.js"></script>
    <button class="toggle">Toggle Points</button>
    <div style="width: 100%; height: 100%">
        <div id="ID" style="width: 100%; height: 100%";></div>
    </div>
    <script type="text/javascript">
        var viewer1 = OpenSeadragon({
            id: "ID",
            prefixUrl: "/openseadragon/images/",
            tileSources: "REPLACE.dzi",

            minZoomImageRatio: 0.5,
            maxZoomImageRatio: Infinity,

            defaultZoomLevel: 1,
            showReferenceStrip: true,
            showNavigator: true,
            // debugMode: true,
        });
        setTimeout(function()
            {
            var dim=viewer1.world.getItemAt(0).getContentSize();
            console.log(dim);
            var width = dim.x;
            var height = dim.y;
            var overlay = viewer1.svgOverlay();
            var selectedIndex = 0;
            $('.toggle').on('click', function() {
                selectedIndex = (selectedIndex + 1) % 2;  
                if ( selectedIndex ) {
                    d3.select(overlay.node()).selectAll("*"). remove();
                } else {
                    var request = new XMLHttpRequest();
                    request.open("GET", 'POINTS', false);
                    request.send(null);
                    var points = JSON.parse(request.responseText);
                    
                    for (var i = 0; i < points.length; i++) {
                        var d3Circle = d3.select(overlay.node()).append("circle")
                    .style('fill', 'rgba(255,0,0,0.2)')
                    .style("stroke", "red")
                    .style("stroke-width", 0.0001)
                    .attr("id", "svg-circle-" + i)
                    .attr("cx",((points[i][1])/width))//, points[i].x)//
                    .attr("cy",((points[i][0])/width))//points[i].y)//
                    .attr("r", points[i][2])//RADIUS
                    .attr("class", "svg-circle")
                    }
                }       
            });
            }, 3000);
    </script>
</body>
</html>""",dict(POINTS=os.path.basename(json_file),
               RADIUS=str(radius),
               REPLACE=wsi_name,
               ID=f"{wsi_name}_{np.random.random_sample()}"))
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO])# ,assets_folder=os.path.abspath("dzi_files")
server = app.server

#3rd port
app_port=5558
#2nd port
slide_server_port=5559

layouts=dict()
elements=dict()
all_app_data=dict(case=None,
                  case_id=None,
                 url=f"http://127.0.0.1:{slide_server_port}",
                 index_html="web_app_test_final/index.html",
                 tmp_aty_df=None,
                 json_file="web_app_test_final/selected_points.json",
                 compression=2.,
                 uro_thresh=None,
                 nc_thresh=None,
                 results_dir="sanity_check_final_v2/")
# sauth --dir web_app_test_final/ user pass localhost 5559

cases=[dict(label=x,value=x) for x in np.vectorize(lambda f: os.path.basename(f).replace(".pkl",""))(glob.glob(os.path.join(all_app_data['results_dir'],"*.pkl"))) if len(glob.glob(f"web_app_test_final/*{x}*.dzi"))>0]

elements['case_lbl']=[dbc.Col(html.Label(x)) for x in ["Case:","Uro Threshold:","NC Threshold:","Max Radius","Scale Radius?","Update WSI Plot:"]]#No Plot Points?

elements['case']=[dbc.Col(x) for x in [
                    dcc.Dropdown(options=cases,value=None,id="case_id", multi=False),
                    dcc.Slider(min=0,max=1, step=1e-2,value=0.5,id="uro_thresh"),
                    dcc.Slider(min=0,max=1, step=1e-2,value=0.1,id="nc_thresh"),
                    dcc.Slider(min=0,max=0.1, step=1e-3,value=0.01,id="max_radius"),
#                     daq.ToggleSwitch(id='malignancy_radius',value=False),
                    dcc.RadioItems(
                        options=[
                            {'label': 'No Scaling', 'value': 'none'},
                            {'label': 'NC Ratio', 'value': 'nc'},
                            {'label': 'Atypia Score', 'value': 'aty'},
                            {'label': 'Urothelial Score', 'value': 'uro'},
                        ],
                        value='none',
                        id='malignancy_radius'
                    ),  
#                     html.Button('No Points', id='disable_pt', n_clicks=0),
                    html.Button('Update', id='update_plt', n_clicks=0)
                    ]
                 ]

layouts['tab1']=dbc.Container([
                    dbc.Row([
                            dbc.Col([
                              html.Div(html.Div(html.Div(html.Iframe(id='openseadragon',
                                        src=all_app_data['url'],width='800',height='800', style=dict(scrolling="no")),id='wsi'),id='wsi2'),id='wsi3'),
                                
                            ]),
#                             dbc.Col([html.Div(html.Iframe(id='osd_2',
#                                         src=all_app_data['url'],width='400',height='400'),id='wsi4')
#                                 ]),
                            dbc.Col([
                              dbc.Row(elements['case_lbl']),
                              dbc.Row(elements['case']),
                              dbc.Row(dcc.Graph(style=dict(width='300',height='300'),id='malignancy_plot'))
                                ])
                            ],no_gutters=True,align="center"),

                    ], 
                    fluid=True)

layouts['tab2']=dbc.Container([
                    html.Div([html.Label("WIP")],id='wip')
                ])

tabs = html.Div(
    [
        dbc.Tabs(
            [
                dbc.Tab(label="Autoparis Results", tab_id="tab-1", children=layouts['tab1']),
                dbc.Tab(label="Individual Cell Images", tab_id="tab-2", children=layouts['tab2']),

            ],
            id="tabs",
            active_tab="tab-1",
        ),
        html.Div(id="content"),
    ]
)

app.layout = tabs

def prevent_update_check(conditional):
    global all_app_data
    if (conditional) or all_app_data['case'] is None: raise PreventUpdate

@app.callback(
    Output("wsi2", "children"),
    [Input('case_id','value')]
)
def update_case(case):
    global all_app_data
    if case == all_app_data['case_id'] or case is None: raise PreventUpdate
    all_app_data['case_id']=case
    all_app_data["case"]=pd.read_pickle(os.path.join(all_app_data["results_dir"],f"{case}.pkl"))
    all_app_data["case"]['cell_stats']['nc']=all_app_data["case"]['nc'][2]/(all_app_data["case"]['nc'][[1,2]].sum(1))
    all_app_data["case"]['cell_stats']['uro']=softmax(all_app_data["case"]['uro_decision'],1)[:,0]
    all_app_data["case"]['cell_stats']['aty']=all_app_data["case"]['atypia']
#     all_app_data["case"]=all_app_data["case"]['cell_stats']
    all_app_data['tmp_aty_df']=None
    with open(all_app_data['index_html'],'w') as f: f.write(return_wsi_file(case,True,all_app_data['json_file'],5e-4,1.,1.))
    return html.Div(html.Iframe(id=f'openseadragon_{np.random.random_sample()}', src=all_app_data['url'],width='800',height='800',style=dict(scrolling="no")),id='wsi')

# @app.callback(
#     Output("wsi2", "children"),
#     [Input('case_id','value'),
#      Input('disable_pt','n_clicks')]
# )
# def points_off(case,n_clicks):
#     global all_app_data
#     raise PreventUpdate
#     prevent_update_check(n_clicks is None or not os.path.exists(all_app_data['index_html']))
#     with open(all_app_data['index_html'],'w') as f: f.write(return_wsi_file(case,False,'',0,0,0))
#     return html.Div(html.Iframe(id=f'openseadragon_{np.random.random_sample()}',
#                 src=all_app_data['url'],width='400',height='400'),id='wsi')

@app.callback(
    Output('malignancy_plot','figure'),
    [Input('uro_thresh','value'),Input('nc_thresh','value')]
)
def draw_aty_scatter(uro_thresh, nc_thresh):
    global all_app_data
    prevent_update_check(not os.path.exists(all_app_data['index_html']))
    all_app_data['tmp_aty_df']=all_app_data["case"]['cell_stats'].loc[(all_app_data["case"]['cell_stats']['uro']>=uro_thresh)&(all_app_data["case"]['cell_stats']['nc']>=nc_thresh)]
    fig=px.scatter(x='nc',y='aty',data_frame=all_app_data['tmp_aty_df'],title='Pan-cellular Plot',
                  labels={
                     "nc": "NC Ratio",
                     "aty": "Atypical Score",
                 })
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        })
    all_app_data['uro_thresh']=uro_thresh
    all_app_data['nc_thresh']=nc_thresh
    return fig

# if either case is changed or points dragged over
@app.callback(
    Output("wip", "children"),
    [
     Input('malignancy_plot','selectedData'),
     Input('update_plt','n_clicks'),
     Input('max_radius','value'),
     Input('malignancy_radius','value')
    ]
)
def plot_selected_points(selected_points,n_clicks,max_radius,malignancy_radius):
    global all_app_data
    prevent_update_check(any([all_app_data.get("uro_thresh",None)==None,n_clicks == None, not os.path.exists(all_app_data['index_html']), selected_points == None, isinstance(all_app_data['tmp_aty_df'],type(None))]))
    #print("here i am")
#     all_app_data['tmp_aty_df']=all_app_data["case"].loc[all_app_data["case"]['uro']>=uro_thresh]
    pts=np.array([pt['pointIndex'] for pt in selected_points['points']])
    #print(pts)
    prevent_update_check(len(pts)==0)
    plot_points=np.array(all_app_data['tmp_aty_df'].iloc[pts]['centroid'].tolist())
    plot_points/=all_app_data['compression']
    plot_points=np.hstack([plot_points,(all_app_data['tmp_aty_df'].iloc[pts][malignancy_radius].values.reshape(-1,1) if malignancy_radius!='none' else np.ones(len(pts)).reshape(-1,1).astype(float))*max_radius ])
#     print(plot_points)
    plot_points=plot_points.tolist()
    with open(all_app_data['json_file'],'w') as f: json.dump(plot_points,f)
    with open(all_app_data['index_html'],'w') as f: f.write(return_wsi_file(all_app_data['case_id'],True,all_app_data['json_file'],5e-4,1.,1.))
    return "updated"
#         html.Div(html.Div(html.Iframe(id=f'openseadragon',# openseadragon_{np.random.random_sample()}
#                 src=all_app_data['url'],width='400',height='400'),id='wsi'),id='wsi2')
           


In [25]:
# sauth --dir web_app_test_final/ user pass localhost 5454

In [26]:
app.run_server(debug=True,mode="external",host='localhost',port=app_port)

Dash app running on http://localhost:5558/


In [4]:
import pandas as pd
pd.__version__

'1.2.3'

In [34]:
# ! cp -al webapp_images_dzi/*.css web_app_test

In [163]:
import cv2
im=cv2.imread("webapp_images/neg_18_4_14_N_orig.png")

In [164]:
im.shape

(11119, 10718, 3)

In [ ]:
# https://github.com/openseadragon/svg-overlay/issues/20
# ! ls webapp_images_dzi/images -t | grep neg_18_4_14_N_orig
from scipy.special import softmax

! for f in $(ls webapp_images_dzi/images -t | grep neg_18_4_14_N_orig) ; do cp -al webapp_images_dzi/images/$f web_app_test/ ; done 
stats=pd.read_pickle("sanity_check2/neg_18_4_14_N_orig.pkl")
stats["uro_decision"]#.keys()
stats['cell_stats']['nc']=stats['nc'][2]/(stats['nc'][[1,2]].sum(1))
stats['cell_stats']['uro']=softmax(stats['uro_decision'],1)[:,0]
stats['cell_stats']['aty']=stats['atypia']